[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [1]:
!pip install kora -q
from kora import drive
drive.link_nbs()

     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
Mounted at /content/drive


In [2]:
from pix import rgb_to_lab

importing Jupyter notebook from /nbs/pix.ipynb


In [3]:
import numpy as np
import tensorflow as tf
from PIL import Image
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [4]:
def make_dataloaders(batch_size=16, im_size=256, val_split= 0.725, split = 'Train', path = '/content/dataset/'):
    if split == 'Train':
        # if training, augment the data
        datagen = ImageDataGenerator(
            rescale = 1/255.,
            shear_range=0.2,
            zoom_range=0.2,
            rotation_range=20,
            horizontal_flip=True,
            validation_split=val_split)
        # import from a directory with subdirectory 'Train', can split for validation
        generator = datagen.flow_from_directory(
            path,
            target_size=(im_size, im_size),
            color_mode='rgb',
            batch_size = batch_size,
            classes = [split],
            subset = 'training',
            class_mode='input',
            shuffle=True)
    elif split == 'Test':
        # no data augmentation, just rescale the image to 0-1
        datagen = ImageDataGenerator(
            rescale = 1/255.,
            )
        # import from a directory with subdirectory 'Test', no shuffle
        generator = datagen.flow_from_directory(
            path,
            target_size=(im_size, im_size),
            color_mode='rgb',
            batch_size = batch_size,
            classes = [split],
            class_mode='input',
            shuffle=False)
    return generator